In [1]:
#!/usr/bin/python3
import os
import json
import re
import csv
from scipy.stats import chi2_contingency
import numpy as np
import xlrd
import pandas as pd

In [2]:
def clean_str(s):  
    s = re.sub('[^0-9\w\d\s]+', '', s).replace('\r'," ").replace('\n'," ").replace('\xa0',' ')
    return s.strip().lower()

In [3]:
Train_Fake_Data = []
Train_True_Data = []

In [4]:
f = open("./data/Fake_train")
for line in f.readlines():
    Train_Fake_Data.append(line)
f.close()

f = open("./data/True_train")
for line in f.readlines():
    Train_True_Data.append(line)
f.close()

In [5]:
ALL_word_dict = {}
ALL_word_list = []

for line in Train_Fake_Data:
    line = line.strip('\n').split(' ')
    for word in line:
        if word not in ALL_word_dict.keys():
            ALL_word_dict[word] = 1
        else:
            ALL_word_dict[word] += 1

for line in Train_True_Data:
    line = line.strip('\n').split(' ')
    for word in line:
        if word not in ALL_word_dict.keys():
            ALL_word_dict[word] = 1
        else:
            ALL_word_dict[word] += 1


for k, v in ALL_word_dict.items():
    if v > 50:
        ALL_word_list.append(k)

print('ALL Word : ', len(ALL_word_list))

ALL Word :  1381


In [6]:
def data_analysis(target_word_name=None):
    # Data Analysis

    word_list = {}

    #  ************** Target Word List ******************
    print("Get "+target_word_name+" Data Begining ......")
   
    target_words_num = {}
    target_words_list = []
    target_words_weight = {}

    if target_word_name == 'all':
        for word in ALL_word_list:
            target_words_num[word] = [0, 0, 0]
            target_words_list.append(word)
            try:
                target_words_weight[word] = '1'
            except:
                target_words_weight[word] = '1'

    elif target_word_name == 'persuasive':
        persuasive_words_list = []
        target_data_1 = open('../other_dict/Persuasive_word.csv', encoding='utf-8', mode='r')
        target_data_1 = target_data_1.readlines()
        for line in target_data_1:
            line = line.strip('\n\r').lower().split('\t')
            target_words_num[line[0]] = [0, 0, 0]
            target_words_list.append(line[0])
            try:
                target_words_weight[line[0]] = line[1]
            except:
                target_words_weight[line[0]] = '1'

        target_data_2 = open('../other_dict/Persuasive_phase.csv', encoding='utf-8', mode='r')
        target_data_2 = target_data_2.readlines()
        for line in target_data_2:
            line = line.strip('\n\r').lower()
            persuasive_words_list.append(line)
            target_words_num[line] = [0, 0, 0]
            target_words_list.append(line)

            try:
                target_words_weight[line] = '1'
            except:
                target_words_weight[line] = '1'


    elif target_word_name == 'sentiment':
        target_data = open('../other_dict/AFINN-en-165.txt', encoding='utf-8', mode='r')
        target_data = target_data.readlines()
        for line in target_data:
            line = line.strip('\n\r').lower().split('\t')
            target_words_num[line[0]] = [0, 0, 0]
            target_words_list.append(line[0])
            try:
                target_words_weight[line[0]] = line[1]
            except:
                target_words_weight[line[0]] = '1'
                
    elif target_word_name == 'nrc_emotion':
        target_data = open('../true_fake_analysis/result_fake_true_after_reduce_stopwords/new_all_nrc_emotion_words.csv', encoding='utf-8', mode='r')
        target_data = target_data.readlines()
        for line in target_data:
            line = line.strip('\n\r').lower().split(',')
            target_words_num[line[0]] = [0, 0, 0]
            target_words_list.append(line[0])
            try:
                target_words_weight[line[0]] = line[1]
            except:
                target_words_weight[line[0]] = '1'

                
    elif target_word_name == 'subjectivity':
        target_data = open('../other_dict/subjclueslen1-HLTEMNLP05.csv', encoding='utf-8', mode='r')
        target_data = target_data.readlines()
        for line in target_data:
            line = line.strip('\n\r').lower().split(',')
            target_words_num[line[2]] = [0, 0, 0]
            target_words_list.append(line[2])
            try:
                target_words_weight[line[2]] = line[0] + '+' + line[5]
            except:
                target_words_weight[line[2]] = '1'

    elif target_word_name == 'technical':

        workbook = xlrd.open_workbook('../other_dict/acadCore.xlsx')
       
        booksheet = workbook.sheet_by_name('list') 
      
        for i in range(3016):
            word = booksheet.cell_value(i, 1)
            target_words_num[word] = [0, 0, 0]
            target_words_list.append(word)
            try:
                target_words_weight[word] = '1'
            except:
                target_words_weight[word] = '1'

    elif target_word_name == 'propaganda':

        fake_pro_data = []
        true_pro_data = []

        target_data_1 = open('../other_dict/task-1/task1.train.txt', encoding='utf-8', mode='r')
        target_data_1 = target_data_1.readlines()
        for line in target_data_1:
            line = line.strip('\n\r').lower().split('\t')
            text = line[0]
            label = line[2]
            if label == 'propaganda':
                true_pro_data.append(text)
            else:
                fake_pro_data.append(text)
        tesk_2_3_path = '../other_dict/tasks-2-3/train/'
        target_data_2_dirs = os.listdir(tesk_2_3_path)
        article_index_list = []
        for file_name in target_data_2_dirs:
            an_article_index = file_name[:16]
            article_index_list.append(an_article_index)

        for index in article_index_list:
            text_name = tesk_2_3_path + index + '.txt'
            try:
                text = open(text_name, encoding='utf-8', mode='r')
            except:
                print('No such file or directory: ', text_name)
                continue

            try:
                label_name = tesk_2_3_path + index + '.task2.labels'
                label = open(label_name, encoding='utf-8', mode='r')
            except:
                label_name = tesk_2_3_path + index + '.task3.labels'
                label = open(label_name, encoding='utf-8', mode='r')
            text = text.readlines()
            text_list = []
            for line in text:
                text_list.append(line)
            label = label.readlines()
            label_list = []
            for line in label:
                label_list.append(line[2])

            for i in range(len(text_list)):
                if i < len(label_list):
                    if text_list[i] != '':
                        if label_list[i] == 'propaganda':
                            true_pro_data.append(text_list[i])
                        elif label_list[i] == 'non-propaganda':
                            fake_pro_data.append(text_list[i])
                        # else:
                            # print(label_list[i])
                            # print("No Match !!!", index, i)
        fake_pro_num = 0 
        true_pro_num = 0

        word_list = {}

       
        for line in fake_pro_data:
            line_word_list = []
            fake_pro_num += 1
            line = line.strip('\n').split(' ')
            for word in line:
                if word in line_word_list:
                    continue
                else:
                    line_word_list.append(word)
                    if word in word_list.keys():
                        word_list[word][0] += 1
                    else:
                        word_list[word] = [1, 0]

        for line in true_pro_data:
            line_word_list = []
            true_pro_num += 1
            line = line.strip('\n').split(' ')
            for word in line:
                if word in line_word_list:
                    continue
                else:
                    line_word_list.append(word)
                    if word in word_list.keys():
                        word_list[word][1] += 1
                    else:
                        word_list[word] = [0, 1]

        print("fake_pro_num, true_pro_num ",fake_pro_num, true_pro_num)

        fake_word_dict = {}
        true_word_dict = {}

     
        for word, v in word_list.items():
            word_in_fake = v[0]
            word_in_true = v[1]
            if word_in_fake + word_in_true > 5:
                # print(word_in_fake, word_in_true)
                word_not_in_fake = fake_pro_num - word_in_fake
                word_not_in_true = true_pro_num - word_in_true
                obs = np.array([[word_in_fake, word_not_in_fake], [word_in_true, word_not_in_true]])
                g, p, dof, expctd = chi2_contingency(obs)
                if(p < 0.05):  
                    if word_in_fake > word_in_true:
                        fake_word_dict[word] = p
                    else:
                        true_word_dict[word] = p

        propaganda_word_list = []
        print("fake_word_dict",len(fake_word_dict))
        print("true_word_dict",len(true_word_dict))
        for w, p in fake_word_dict.items():
            propaganda_word_list.append(w)
        for w, p in true_word_dict.items():
            propaganda_word_list.append(w)

        for word in propaganda_word_list:
            target_words_num[word] = [0, 0, 0]
            target_words_list.append(word)
            try:
                target_words_weight[word] = '1'
            except:
                target_words_weight[word] = '1'

    #  **************************************************

    fake_data = Train_Fake_Data
    true_data = Train_True_Data


    fake_num = len(Train_Fake_Data) 
    true_num = len(Train_True_Data)

    print('fake_num, true_num:', fake_num, true_num)

    fake_word_list = []
    true_word_list = []

    if target_word_name != 'persuasive':
        for line in fake_data:
            line = line.strip('\n').split(' ')
            for word in line:
                if word not in fake_word_list:
                    fake_word_list.append(word)

                if word in target_words_list:
                    target_words_num[word][0] += 1
        print('OK_fake_data')
        for line in true_data:
            line = line.strip('\n').split(' ')
            for word in line:
                if word not in true_word_list:
                    true_word_list.append(word)

                if word in target_words_list:
                    target_words_num[word][1] += 1
        print('OK_true_data')


    else:
        for line in fake_data:
            line_1 = line.strip('\n').split(' ')
            for word in line_1:
                if word not in fake_word_list:
                    fake_word_list.append(word)

                if word in target_words_list:
                    target_words_num[word][0] += 1
            line_2 = line.strip('\n')
            for phase in persuasive_words_list:
                if phase in line_2:
                    if phase not in fake_word_list:
                        fake_word_list.append(phase)
                    target_words_num[phase][0] += 1

        print('OK_fake_data')
        for line in true_data:
            line_1 = line.strip('\n').split(' ')
            for word in line_1:
                if word not in true_word_list:
                    true_word_list.append(word)

                if word in target_words_list:
                    target_words_num[word][1] += 1
            line_2 = line.strip('\n')
            for phase in persuasive_words_list:
                if phase in line_2:
                    if phase not in true_word_list:
                        true_word_list.append(phase)
                    target_words_num[phase][1] += 1
        print('OK_true_data')



  
    fake_word_num = len(fake_word_list)
    true_word_num = len(true_word_list)


    fake_word_dict = {}
    true_word_dict = {}


   
    for word, v in target_words_num.items():
        word_in_fake = v[0]
        word_in_true = v[1]


        if word_in_fake + word_in_true > 10:
            # print(word_in_fake, word_in_true, word_in_satire)
            word_not_in_fake = fake_num - word_in_fake
            word_not_in_true = true_num - word_in_true
           
            obs = np.array([[word_in_fake, word_in_true],
                            [word_not_in_fake, word_not_in_true]])
            try:
                chi2, p, dof, expected = chi2_contingency(obs)
               
                if(p < 0.05): 

                    if (word_in_fake > word_in_true) :  
                        fake_word_dict[word] = p
                    elif (word_in_true > word_in_fake) :  
                        true_word_dict[word] = p

            except:
#                 print(word, obs)

                '''
                if word_in_fake > word_in_true: 
                    fake_word_dict[word] = word_in_fake - word_in_true
                elif word_in_fake < word_in_true:
                    true_word_dict[word] = word_in_true - word_in_fake
                else:
                    neutral_word_dict[word] = 0
                '''

    # print(fake_word_dict, true_word_dict, neutral_word_dict)

    fake_word_dict = sorted(fake_word_dict.items(), key=lambda d: d[1], reverse=False) 
    true_word_dict = sorted(true_word_dict.items(), key=lambda d: d[1], reverse=False)


   
    #fake_target_words_path = './result_fake_true_new/fake_' + target_word_name + '_words.csv'
    fake_target_words_path = './result_fake_true_2910/fake_' + target_word_name + '_words.csv'
    with open(fake_target_words_path, "w") as fake_key_words:
        writer = csv.writer(fake_key_words)
        for i, item in enumerate(fake_word_dict):
            clean_word = clean_str(item[0])
            try:
                word_weight = target_words_weight[clean_word]
            except:
                word_weight = 'None'
            writer.writerow([clean_word, item[1], word_weight])

    #true_target_words_path = './result_fake_true_new/true_' + target_word_name + '_words.csv'
    true_target_words_path = './result_fake_true_2910/true_' + target_word_name + '_words.csv'
    with open(true_target_words_path, "w") as true_key_words:
        writer = csv.writer(true_key_words)
        for i, item in enumerate(true_word_dict):
            clean_word = clean_str(item[0])
            try:
                word_weight = target_words_weight[clean_word]
            except:
                word_weight = 'None'
            writer.writerow([clean_word, item[1], word_weight])


#     print(target_words_weight)

    print("Get "+target_word_name+" Data End !!!")

In [8]:
data_analysis('all')
data_analysis('persuasive')
data_analysis('sentiment')
data_analysis('nrc_emotion')
data_analysis('subjectivity')
data_analysis('technical')
data_analysis('propaganda')


Get all Data Begining ......
fake_num, true_num: 1921 2893
OK_fake_data
OK_true_data
Get all Data End !!!
Get persuasive Data Begining ......
fake_num, true_num: 1921 2893
OK_fake_data
OK_true_data
Get persuasive Data End !!!
Get sentiment Data Begining ......
fake_num, true_num: 1921 2893
OK_fake_data
OK_true_data
Get sentiment Data End !!!
Get nrc_emotion Data Begining ......
fake_num, true_num: 1921 2893
OK_fake_data
OK_true_data
Get nrc_emotion Data End !!!
Get subjectivity Data Begining ......
fake_num, true_num: 1921 2893
OK_fake_data
OK_true_data
Get subjectivity Data End !!!
Get technical Data Begining ......
fake_num, true_num: 1921 2893
OK_fake_data
OK_true_data
Get technical Data End !!!
Get propaganda Data Begining ......
No such file or directory:  ../other_dict/tasks-2-3/train/article765197039.txt
No such file or directory:  ../other_dict/tasks-2-3/train/article766632016.txt
No such file or directory:  ../other_dict/tasks-2-3/train/article765197039.txt
No such file or dir

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/linliu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
from nltk.corpus import stopwords
words = stopwords.words('english')
print(words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
import os
path = "./result_fake_true_2910/" 
files= os.listdir(path)
s = []
for file in files: 
     if not os.path.isdir(file): 
        print("now...",file)
        if(file == '.DS_Store'):
            continue
        f = open(path+"/"+file); 
        iter_f = iter(f);
        fw = open("./result_fake_true_after_reduce_stopwords/"+file,'w+')
        reduce_list = []
        for line in iter_f: 
            word = line.split(',')[0]
            if word not in words:
                fw.write(line)
            else:
                reduce_list.append(word)
        print(reduce_list)
        f.close()
        fw.close()

now... true_propaganda_words.csv
['her', 'i', 'with', 'it', 'this', 'not', 'be', 'are', 'she', 'they', 'from', 'has', 'have', 'no', 'by', 'been', 'if', 'you', 'than', 'an', 'all', 'after', 'out', 'now', 'me', 'more', 'at', 'while', 'them', 'or', 'o', 'itself', 'him', 'only', 'both', 'will', 'being', 'can', 'as', 'below', 'up', 'over', 'under', 'because', 'down', 'doing', 'about', 'myself', 'once', 'any', 'but', 'whom', 'which', 'had', 'again', 'most', 'your', 'do', 'just', 'off', 'its', 'why', 'having']
now... true_nrc_emotion_words.csv
[]
now... true_technical_words.csv
['both']
now... .DS_Store
now... fake_all_words.csv
['their', 'these', 'during', 'should', 'such', 'himself', 'themselves', 'further', 'nor']
now... fake_propaganda_words.csv
['their', 'these', 'during', 'should', 'such', 'himself', 'themselves', 'further', 'nor']
now... fake_persuasive_words.csv
['during', 'should']
now... true_all_words.csv
['her', 'i', 'with', 'it', 'this', 'not', 'be', 'are', 'she', 'they', 'from',

# add wordnet

In [6]:
from nltk.corpus import wordnet as wn

In [7]:
extend_dict = {}
for word in ALL_word_list:
    res = []
    for syssets in wn.synsets(word):
        res.extend(wn.synset(syssets.name()).lemma_names())
    res = list(set(res))
    extend_dict[word]=res

In [8]:
import json
with open('./extend_dict.json','w') as outfile:
    json.dump(extend_dict,outfile,ensure_ascii=False)
    outfile.write('\n')

In [9]:
def data_extend(data):
    res_data = []
    for line in data:
        new_line = []
        for word in line.split(' '):
            new_line.append(word)
            if(word in extend_dict):
                new_line.extend(extend_dict[word])
        res_data.append(' '.join(new_line))
    return res_data

In [10]:
Train_Fake_Data_extend = data_extend(Train_Fake_Data)
Train_True_Data_extend = data_extend(Train_True_Data)

In [13]:
extend_word_dict = {}
extend_word_list = []

for line in Train_Fake_Data_extend:
    line = line.strip('\n').split(' ')
    for word in line:
        
        if word not in extend_word_dict.keys():
            extend_word_dict[word] = 1
        else:
            extend_word_dict[word] += 1

for line in Train_True_Data_extend:
    line = line.strip('\n').split(' ')
    for word in line:
        if word not in extend_word_dict.keys():
            extend_word_dict[word] = 1
        else:
            extend_word_dict[word] += 1

ALL_word_list_extend = []
for k, v in extend_word_dict.items():
    if v > 50:
        ALL_word_list_extend.append(k)

print('ALL Word : ', len(ALL_word_list_extend))

ALL Word :  8534


In [15]:

target_words_num = {}
target_words_list = []
target_words_weight = {}

for word in ALL_word_list_extend:
    target_words_num[word] = [0, 0, 0]
    target_words_list.append(word)
    try:
        target_words_weight[word] = '1'
    except:
        target_words_weight[word] = '1'

fake_data = Train_Fake_Data_extend
true_data = Train_True_Data_extend


fake_num = len(Train_Fake_Data_extend)  
true_num = len(Train_True_Data_extend)

print('fake_num, true_num:', fake_num, true_num)

fake_word_list = []
true_word_list = []


for line in fake_data:
    line = line.strip('\n').split(' ')
    for word in line:
        if word not in fake_word_list:
            fake_word_list.append(word)

        if word in target_words_list:
            target_words_num[word][0] += 1
print('OK_fake_data')
for line in true_data:
    line = line.strip('\n').split(' ')
    for word in line:
        if word not in true_word_list:
            true_word_list.append(word)

        if word in target_words_list:
            target_words_num[word][1] += 1
print('OK_true_data')


# print(persusive_words_num)
fake_word_num = len(fake_word_list)
true_word_num = len(true_word_list)


fake_word_dict = {}
true_word_dict = {}

#     print('OK')


for word, v in target_words_num.items():
    word_in_fake = v[0]
    word_in_true = v[1]


    if word_in_fake + word_in_true > 10:
        # print(word_in_fake, word_in_true, word_in_satire)
        word_not_in_fake = fake_num - word_in_fake
        word_not_in_true = true_num - word_in_true

        obs = np.array([[word_in_fake, word_in_true],
                        [word_not_in_fake, word_not_in_true]])
        try:
            chi2, p, dof, expected = chi2_contingency(obs)
            # if any expected cell counts are < 5, chi2 may not be valid
            # if this is a 2x2, switch to fisher exact
            # print(expected)
            if expected.min() > 5:
#                     print(word, p)
                if p < 0.05:  

                    if (word_in_fake > word_in_true) :  
                        fake_word_dict[word] = p
                    elif (word_in_true > word_in_fake) :  
                        true_word_dict[word] = p

        except:
#                 print(word, obs)

            '''
            if word_in_fake > word_in_true:  
                fake_word_dict[word] = word_in_fake - word_in_true
            elif word_in_fake < word_in_true:
                true_word_dict[word] = word_in_true - word_in_fake
            else:
                neutral_word_dict[word] = 0
            '''

# print(fake_word_dict, true_word_dict, neutral_word_dict)

fake_word_dict = sorted(fake_word_dict.items(), key=lambda d: d[1], reverse=False)  
true_word_dict = sorted(true_word_dict.items(), key=lambda d: d[1], reverse=False)



fake_target_words_path = './result_fake_true_after_reduce_stopwords/fake_' + 'all_extend' + '_words.csv'
with open(fake_target_words_path, "w") as fake_key_words:
    writer = csv.writer(fake_key_words)
    for i, item in enumerate(fake_word_dict):
        clean_word = clean_str(item[0])
        try:
            word_weight = target_words_weight[clean_word]
        except:
            word_weight = 'None'
        writer.writerow([clean_word, item[1], word_weight])

true_target_words_path = './result_fake_true_after_reduce_stopwords/true_' + 'all_extend' + '_words.csv'
with open(true_target_words_path, "w") as true_key_words:
    writer = csv.writer(true_key_words)
    for i, item in enumerate(true_word_dict):
        clean_word = clean_str(item[0])
        try:
            word_weight = target_words_weight[clean_word]
        except:
            word_weight = 'None'
        writer.writerow([clean_word, item[1], word_weight])


#     print(target_words_weight)

fake_num, true_num: 1921 2893
OK_fake_data
OK_true_data
